In [ ]:
###################Encryption, Update and Keystream function definition Start#####################

def keystream(S,B):                                       #Keystream bit generation function
    L = B[7] ^ B[11] ^ B[30] ^ B[40] ^ B[45] ^ B[54] ^ B[71]
    Q = B[4]&B[21] ^ B[9]&B[52] ^ B[18]&B[37] ^ B[44]&B[76]
    T = B[5] ^ B[8]&B[82] ^ B[34]&B[67]&B[73] ^ B[2]&B[28]&B[41]&B[65] ^ B[13]&B[29]&B[50]&B[64]&B[75] \
    ^ B[6]&B[14]&B[26]&B[32]&B[47]&B[61] ^ B[1]&B[19]&B[27]&B[43]&B[57]&B[66]&B[78]
    T_c = S[23] ^ S[3]&S[16] ^ S[9]&S[13]&B[48] ^ S[1]&S[24]&B[38]&B[63]
    
    out = L ^ Q ^ T ^ T_c
    return out

def NFSR1_bk_update(S):                                   #Feedback function for NFSR1 in backward direction
    temp = S[30]  ^ S[1]  ^  S[4]  ^ S[5]  ^  S[14] ^ S[16] ^ S[17] ^ S[19] ^  S[24] ^ S[7]&S[17] ^ S[7]&S[19] ^ S[11]&S[20] \
        ^ S[13]&S[18] ^ S[16]&S[20] ^ S[19]&S[21] ^ S[3]&S[11]&S[21] ^ S[3]&S[18]&S[21] ^ S[6]&S[19]&S[20] \
        ^ S[7]&S[17]&S[21] ^ S[7]&S[19]&S[21] ^ S[11]&S[18]&S[21] ^ S[19]&S[20]&S[21] ^ S[3]&S[6]&S[11]&S[20]\
        ^ S[3]&S[6]&S[18]&S[20] ^ S[3]&S[11]&S[20]&S[21] ^ S[3]&S[18]&S[20]&S[21] ^ S[6]&S[7]&S[17]&S[20] \
        ^ S[6]&S[7]&S[19]&S[20] ^ S[6]&S[11]&S[18]&S[20] ^ S[7]&S[17]&S[20]&S[21] ^ S[7]&S[19]&S[20]&S[21]\
        ^ S[11]&S[18]&S[20]&S[21]
    return temp

def NFSR2_bk_update(B,s0):                                #Feedback function for NFSR2 in backward direction
    temp = s0 ^ B[89] ^ B[23] ^ B[48] ^ B[78] ^ B[83] ^ B[2]&B[58] ^ B[9]&B[11] ^ B[14]&B[15] ^ B[24]&B[52] \
    ^ B[34]&B[41] ^ B[54]&B[57] ^ B[59]&B[73] ^ B[19]&B[21]&B[22] ^ B[61]&B[67]&B[71] \
    ^ B[76]&B[79]&B[80]&B[82]
    
    return temp
    
def NFSR1_fw_update(S):                                   #Feedback function for NFSR1 in forward direction
    temp = S[0]  ^ S[2]  ^  S[5]  ^ S[6]  ^  S[15] ^ S[17] ^ S[18] ^ S[20] ^  S[25] ^ S[8]&S[18] ^ S[8]&S[20] ^ S[12]&S[21] \
        ^ S[14]&S[19] ^ S[17]&S[21] ^ S[20]&S[22] ^ S[4]&S[12]&S[22] ^ S[4]&S[19]&S[22] ^ S[7]&S[20]&S[21] \
        ^ S[8]&S[18]&S[22] ^ S[8]&S[20]&S[22] ^ S[12]&S[19]&S[22] ^ S[20]&S[21]&S[22] ^ S[4]&S[7]&S[12]&S[21]\
        ^ S[4]&S[7]&S[19]&S[21] ^ S[4]&S[12]&S[21]&S[22] ^ S[4]&S[19]&S[21]&S[22] ^ S[7]&S[8]&S[18]&S[21] \
        ^ S[7]&S[8]&S[20]&S[21] ^ S[7]&S[12]&S[19]&S[21] ^ S[8]&S[18]&S[21]&S[22] ^ S[8]&S[20]&S[21]&S[22]\
        ^ S[12]&S[19]&S[21]&S[22]
    
    return temp

    
def NFSR2_fw_update(B,s0):                                #Feedback function for NFSR2 in forward direction
    temp = s0 ^ B[0] ^ B[24] ^ B[49] ^ B[79] ^ B[84] ^ B[3]&B[59] ^ B[10]&B[12] ^ B[15]&B[16] ^ B[25]&B[53] \
    ^ B[35]&B[42] ^ B[55]&B[58] ^ B[60]&B[74] ^ B[20]&B[22]&B[23] ^ B[62]&B[68]&B[72] \
    ^ B[77]&B[80]&B[81]&B[83]
    
    return temp


def Encryption():                                          #Encryption Function
    #Initialisation Phase I
    import random
    K = [random.randint(0,1) for i in range(120)]          #Secret Key
    IV = [random.randint(0,1) for i in range(64)]          #Initialisation Vector

    S = [0]*31                                          #NFSR 1
    B = [0]*90                                          #NFSR 2

    #Phase 1: Key and IV Loading
    for i in range(90):
        if i<=63:
            B[i] = (K[i] ^ IV[i])
        else:
            B[i] = K[i]

    for i in range(29):
        S[i] = K[i + 90]

    S[29] = (K[119] ^ 1)
    S[30] = 1


    #Phase 2: Grain-like Mixing
    for i in range(128):
        z = keystream(S,B)
        b = NFSR2_fw_update(B,S[0])
        s = NFSR1_fw_update(S)
        
        for k in range(89):
            B[k] = B[k+1]
        B[89] = b^z

        for k in range(30):
            S[k] = S[k+1]
        S[30] = s^z

    #Phase 3: Second Key Addition

    for i in range(90):
        B[i] = (B[i]^K[i])
    for i in range(30):
        S[i] = (S[i]^K[90+i])
    S[30] = 1
    
    Z_c = [0]*128


    #Phase 4: Final Diffusion
    for i in range(128):
        Z_c[i] = keystream(S,B)
        b = NFSR2_fw_update(B,S[0])
        s = NFSR1_fw_update(S)

        for k in range(89):
            B[k] = B[k+1]
        B[89] = b

        for k in range(30):
            S[k] = S[k+1]
        S[30] = s

    #Pseudorandom phase
    N_round = 100                #Targeted round in psuedorandom phase (any random number)
    for i in range(N_round):
       # z = keystream(S,B)
        b = NFSR2_fw_update(B,S[0])
        s = NFSR1_bk_update(S)

        for k in range(89):
            B[k] = B[k+1]
        B[89] = b

        for k in range(30):
            S[k] = S[k+1]
        S[30] = s
        
    return S,B
####################Encryption, Update and Keystream function definition End#####################



#Lizard Encryption
#Specificaton
#121-bit internal state
#NFSR1 31-bit  NFSR2 90-bit
#NFSR1 S       NFSR2 B
#Key 120-bit
#IV 64-bit

import math
import random
import time


state_len = 121      #state_size
N_round = 100            #Total number of rounds needed
r = [31, 90]         #Size of the registers NFSR 1 (S) and NFSR 2 (B)


bitlen = math.ceil(math.log(state_len,2)) + 1   #Number of bits that is needed to represent the max. HD

N_round_f = math.ceil(N_round/2)                #No. of forward rounds
N_round_b = math.floor(N_round/2)               #No. of backward rounds

print("HD/Reg model Lizard")
print("Number of rounds in forward ", N_round_f," and in backward ", N_round_b)
print("Total number of rounds",N_round)

###############Lizard cipher with a random key and IV to generate keystream and HD ####################################

S_org, B_org = Encryption()         #Internal state picked from the pseudorandom phase
 

S = S_org[:]
B = B_org[:]

Z_f = [0]*N_round_f              #Array to store the forward keystream bits
Z_b = [0]*N_round_b              #Array to store the backward keystream bits

#Array to store the HD of registers (consecutive rounds) in forward directions
hamm_dist_S_f = [0 for i in range(N_round_f-1)]           #HD for register S
hamm_dist_B_f = [0 for i in range(N_round_f-1)]           #HD for register B

#Array to store the HD of registers (consecutive rounds) in backward directions
hamm_dist_S_b = [0 for i in range(N_round_b)]             #HD for register S
hamm_dist_B_b = [0 for i in range(N_round_b)]             #HD for register B


#Information on. keystream and HD/Reg in backward direction
for i in range(N_round_b): 
    s = NFSR1_bk_update(S)            #Feedback function of NFSR 1
    b = NFSR2_bk_update(B,s)          #Feedback function of NFSR 2
    
    temp1 = B + S                 #Temporary variable to store the state before update

    S = [s] + S[:r[0]-1]                  #State update
    B = [b] + B[:r[1]-1] 

    temp2 = B + S                      #Temporary variable to store the state after update
    
    Z_b[i] = keystream(S,B)            #Storing the keystream bits at each round in backward direction

    #Storing the HD of internal  states in backward direction
    for l in range(state_len):
        if l < r[1]:
            hamm_dist_B_b[i] += (temp1[l]^temp2[l])            #HD for register B
        else:
            hamm_dist_S_b[i] += (temp1[l]^temp2[l])            #HD for register S
    
            
    
            
S = S_org[:]
B = B_org[:]
#Information on keystream and HD/Reg in forward direction
for i in range(N_round_f):
    Z_f[i] = keystream(S,B)            #Storing the keystream bits at each round in forward direction
    
    temp1 = B + S                      #Internal state before update
    
    if i < N_round_f -1:
        #updating the registers
        s = NFSR1_fw_update(S)
        b = NFSR2_fw_update(B,S[0])
        
        S = S[1:r[0]] + [s]
        B = B[1:r[1]] + [b]
        
        temp2 = B + S                  #Internal state after update
        
        #Storing the HD of consecutive internal states in forward direction
        for l in range(state_len):
            if l < r[1]:
                hamm_dist_B_f[i] += (temp1[l]^temp2[l])        #HD for register B
            else:
                hamm_dist_S_f[i] += (temp1[l]^temp2[l])        #HD for register S

###############SMT Modelling to recover state bits from keystream and HD/Reg#################
from z3 import *
m = Solver()

#Defining variables of the format BitVec(.,1)
S_var_org = [BitVec('s%d'%i,1) for i in range(r[0])]          #For register S
B_var_org = [BitVec('b%d'%i,1) for i in range(r[1])]          #For register B

S_f = [BitVec('S_f%d'%i,1) for i in range(1,N_round_f,1)]       #Dummy variables for register S in forward direction
B_f = [BitVec('B_f%d'%i,1) for i in range(1,N_round_f,1)]       #Dummy variables for register B in forward direction


S_b = [BitVec('S_b%d'%i,1) for i in range(1,N_round_b+1,1)]     #Dummy variables for register S in backward direction
B_b = [BitVec('B_b%d'%i,1) for i in range(1,N_round_b+1,1)]     #Dummy variables for register B in backward direction




S = S_var_org[:]
B = B_var_org[:]


#To check the satisfiability of equations guess all variables otherwise guess as per the requirement
#S_org, B_org are the original register of the targeted internal state
#Guess accordingly 

#print("The guessed bits are ")
#print("For register S ",end=" = ")
#for i in range(r[0]):
#    print(i,end=", ")
#    m.add(S[i] == S_org[i])
#print("\n For register B ",end=" = ")
#for i in range(r[1]):
#    print(i,end=", ")
#    m.add(B[i] == B_org[i])
#print("\n")


#Array to store the keystream equations    
Z_f_equ = [0]*N_round_f        
Z_b_equ = [0]*N_round_b

#Array to store the dummy variable equations for both register S and B
S_f_equ = [0]*(N_round_f-1)
B_f_equ = [0]*(N_round_f-1)

S_b_equ = [0]*N_round_b
B_b_equ = [0]*N_round_b

#Array to store the HD/Reg equations
hamm_dist_S_f_equ = [0 for i in range(N_round_f-1)] 
hamm_dist_B_f_equ = [0 for i in range(N_round_f-1)] 

hamm_dist_S_b_equ = [0 for i in range(N_round_b)] 
hamm_dist_B_b_equ = [0 for i in range(N_round_b)] 



#Equation generation steps for backward internal states
for i in range(N_round_b):
    
    #Dummy Variable equated to the feedback function
    s = NFSR1_bk_update(S)
    b = NFSR2_bk_update(B,s)
        
    S_b_equ[i] = (S_b[i] == s)          
    B_b_equ[i] = (B_b[i] == b)
    
    temp1 = B + S                              #State before update
    #State updated with dummy variables
    S = [S_b[i]] + S[:r[0]-1]
    B = [B_b[i]] + B[:r[1]-1]

    
    temp2 = B + S                             #State after update
    #Hamming distance equation
    for l in range(state_len):
        if l < r[1]:
            hamm_dist_B_b_equ[i] += ZeroExt(bitlen-1,(temp1[l]^temp2[l]))        #HD equation for register B
        else:
            hamm_dist_S_b_equ[i] += ZeroExt(bitlen-1,(temp1[l]^temp2[l]))        #HD equation for register S
        
    m.add(hamm_dist_B_b_equ[i] == hamm_dist_B_b[i])
    m.add(hamm_dist_S_b_equ[i] == hamm_dist_S_b[i])
        
    #Keystream equation
    Z_b_equ[i] = (keystream(S,B) == Z_b[i])

    
S = S_var_org[:]
B = B_var_org[:]
#Equations generation steps for forward internal states
for i in range(N_round_f):
    Z_f_equ[i] = (keystream(S,B) == Z_f[i])             #Keystream equation
    
    temp1 = B + S                           #State before update
    
    
    if i < N_round_f-1:
        s = NFSR1_fw_update(S)
        b = NFSR2_fw_update(B,S[0])
        
        S_f_equ[i] = (S_f[i] == s)            #Dummy variable equated to the feedback function
        B_f_equ[i] = (B_f[i] == b)
        
        #Internal state updated with dummy variable
        S = S[1:] + [S_f[i]]
        B = B[1:] + [B_f[i]]
        
        temp2 = B + S                       #State after update
        
        #Hamming distance equation
        for l in range(state_len):
            if l < r[1]:
                hamm_dist_B_f_equ[i] += ZeroExt(bitlen-1,(temp1[l]^temp2[l]))          #HD equation for register B
            else: 
                hamm_dist_S_f_equ[i] += ZeroExt(bitlen-1,(temp1[l]^temp2[l]))          #HD equation for register S

        m.add(hamm_dist_B_f_equ[i] == hamm_dist_B_f[i])
        m.add(hamm_dist_S_f_equ[i] == hamm_dist_S_f[i])                #Feeding it to the solver


# All Keystream and Dummy variable equations
Equ = Z_f_equ + Z_b_equ + S_f_equ + B_f_equ + S_b_equ + B_b_equ


m.add(Equ)              #Equation feeded to the model m
start = time.time()
temp = m.check()        #Check the satisfiability of the given system of equations
end = time.time()

#if Satisfiable
if temp == sat:         
    print(temp)
    soln = m.model()           #soln contains the output
    print("SMT Time = ", end - start)         #Time taken by solver to solve
    
    #Extracting the value of the internal state variables
    out_S = [0]*(r[0])
    out_B = [0]*(r[1])
    for i in range(r[0]):
        out_S[i] = soln[S_var_org[i]]
    for i in range(r[1]):
        out_B[i] = soln[B_var_org[i]]
    
    #Check whether the original state is same as the output state
    #S_org, B_org are the original registers whereas out_S and out_B are the output of SMT
    if S_org == out_S and B_org == out_B:
        print("matched")
    else:
        print("############################not matched##############################################")
        print(S_org)
        print(B_org)
        print(out_S)
        print(out_B)
    
#If the solution is Unsatisfiable, check code and equations
else:
    print("###################################unsat###################################################")
    print(temp)
    print(S_org)
    print(B_org)